In [1]:
import pandas as pd
import numpy as np

import gensim
from gensim.models.fasttext import FastText

import re
from string import printable
from razdel import tokenize

from datetime import datetime

In [2]:
df = pd.read_csv('ruscorpora_content.csv', sep=";")
df1 = pd.read_csv('ruscorpora_content1.csv', sep=";")
df2 = pd.read_csv('ruscorpora_content2.csv', sep=";")
df3 = pd.read_csv('ruscorpora_content3.csv', sep=";")
df = pd.concat([df, df1, df2, df3])

In [3]:
df['Publ_year'] = pd.to_datetime(df['Publ_year'], errors='coerce')
df["Publ_year"] = df["Publ_year"].dt.year
df = df[df["Publ_year"] > 1980]

In [4]:
df.shape

(10058, 19)

In [5]:
printable += 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'.upper()

def preprocessing_text(text: str) -> str:
    
    text = text.lower()
    text = text.replace(u"\u00A0", " ")
    text = re.sub('&nbsp;', ' ', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('[?+=);:(\[\]#\'"°~*→»«|!_\t\.\,]', ' ', text)
    text = re.sub("[^{}]+".format(printable), "", text)
    return text

In [6]:
texts = [i for i in df['Full context']]

correctness_texts = []
for text in texts:
    correctness = preprocessing_text(text)
    correctness_texts.append(correctness)
    
df['Full context'] = correctness_texts
df['Full context'] = df['Full context'].apply(lambda x: [a.text for a in tokenize(x)])

In [7]:
texts_model = []
for text in df['Full context']:
    texts_model.extend(text)

In [26]:
%%time
ft_model = FastText(texts_model,
                    vector_size=300,
                    window=10,
                    min_count=5,
                    sg=1)

CPU times: user 8.25 s, sys: 1.11 s, total: 9.36 s
Wall time: 6.39 s


In [27]:
wv = ft_model.wv

### Использованный текст

Мне отвели маленькую комнатку, в четвертом этаже отеля. Здесь известно, что я принадлежу к свите генерала. По всему видно, что они успели-таки дать себя знать. Генерала считают здесь все богатейшим русским вельможей. Еще до обеда он успел, между другими поручениями, дать мне два тысячефранковых билета разменять. Я разменял их в конторе отеля. Теперь на нас будут смотреть, как на миллионеров, по крайней мере целую неделю. Я хотел было взять Мишу и Надю и пойти с ними гулять, но с лестницы меня позвали к генералу; ему заблагорассудилось осведомиться, куда я их поведу. Этот человек решительно не может смотреть мне прямо в глаза; он бы и очень хотел, но я каждый раз отвечаю ему таким пристальным, то есть непочтительным взглядом, что он как будто конфузится. В весьма напыщенной речи, насаживая одну фразу на другую и наконец совсем запутавшись, он дал мне понять, чтоб я гулял с детьми где-нибудь, подальше от воксала, в парке. Наконец он рассердился совсем и круто прибавил

In [28]:
text = 'Мне отвели маленькую комнатку, в четвертом этаже отеля. Здесь известно, что я принадлежу к свите генерала. По всему видно, что они успели-таки дать себя знать. Генерала считают здесь все богатейшим русским вельможей. Еще до обеда он успел, между другими поручениями, дать мне два тысячефранковых билета разменять. Я разменял их в конторе отеля. Теперь на нас будут смотреть, как на миллионеров, по крайней мере целую неделю. Я хотел было взять Мишу и Надю и пойти с ними гулять, но с лестницы меня позвали к генералу; ему заблагорассудилось осведомиться, куда я их поведу. Этот человек решительно не может смотреть мне прямо в глаза; он бы и очень хотел, но я каждый раз отвечаю ему таким пристальным, то есть непочтительным взглядом, что он как будто конфузится. В весьма напыщенной речи, насаживая одну фразу на другую и наконец совсем запутавшись, он дал мне понять, чтоб я гулял с детьми где-нибудь, подальше от воксала, в парке. Наконец он рассердился совсем и круто прибавил:'

text = preprocessing_text(text)
text = [a.text for a in tokenize(text)]

vectors = [wv[feat] for feat in text]

In [29]:
word = 'парке'

feat_vector = [wv[word]]

In [30]:
def cosine_similarities(vector_feat, vectors_all):
    vector_1 = np.array(vector_feat)
    norm = np.linalg.norm(vector_1, axis=1)
    all_norms = np.linalg.norm(vectors_all, axis=1)
    norm_matr = np.dot(np.reshape(all_norms, (all_norms.shape[0], 1)), np.reshape(norm, (1, norm.shape[0])))
    dot_products = np.dot(vectors_all, vector_1.transpose())
    similarities = dot_products / norm_matr
    return 1 - similarities

In [31]:
cos_scores = cosine_similarities(feat_vector, vectors)

In [32]:
if cos_scores.min() < 0.5:
    print(f'True - min cos_scores = {cos_scores.min():.3f}')
else:
    print('False')

True - min cos_scores = 0.000
